In [1]:
import numpy as np

In [8]:
def Backus(V,lamda, G ):
    """Computes stiffnesses of a layered medium using backus average model. Written by Jiaxin Yu (July 2021)

    Args:
        V (num or array-like, frac): volumetric fractions of N isotropic layering materials
        lamda (num or array-like): Lamé coefficients of N isotropic layering materials
        G (num or array-like, GPa): shear moduli of N isotropic layering materials
    Returns:
        C11,C33,C13,C44,C66 (num or array-like, GPa): Elastic moduli of the anisotropic layered media
    """    
    if isinstance(V, ( np.ndarray)) is False:
        V=np.array(V)
    V=V/np.sum(V)
    if isinstance(lamda, ( np.ndarray)) is False:
        lamda=np.array(lamda)
    if isinstance(G, ( np.ndarray)) is False:
        G=np.array(G)
    
    C33=np.dot(V, 1/(lamda+2*G)) **-1
    C44=np.dot(V, 1/G)**-1
    C66=np.dot(V, G)
    C13=np.dot(V, 1/(lamda+2*G)) **-1 * np.dot(V, lamda/(lamda+2*G))
    C11=np.dot(V, 4*G*(lamda+G)/(lamda+2*G))+np.dot(V, 1/(lamda+2*G))**-1 * np.dot(V, lamda/(lamda+2*G))**2
    
    return C11,C33,C13,C44,C66

def Backus_log(Vp,Vs,Den,Depth):
    """ Computes Backus Average from log data, notice that the Depth is 1d Vector including each top depth of layer and also the bottom of last layer. 

    Args:
        Vp (array): P wave velocities of layers [Vp1,Vp2...Vpn], size N
        Vs (array): S wave velocities of layers [Vs1,Vs2...Vsn], size N
        Den (array): Densities of layers, size N
        Depth (array): 1d depth, ATTENTION: each depth point
        corresponds to the top of thin isotropic layer, the bottom of the sedimentary package is the last depth point. [dep1,dep2,,,,,,depn, depn+1], size N+1

    Returns:
        _type_: _description_
    """    

    # compute lame constants from log data 
    G = Den*Vs**2
    lamda = Den*Vp**2 - 2*G
    # compute thickness of each layer given depth  
    thickness = np.diff(Depth)
    # Volume fraction
    V= thickness/thickness.sum()
    
    C11,C33,C13,C44,C66 = Backus(V,lamda, G )

    # Mass balance for density 
    den = np.dot(V, Den)

    return C11,C33,C13,C44,C66, den


In [23]:
def write_VTI_matrix(C11,C33,C13,C44,C66):
    """formulate VTI stiffness matrix 

    Args:
        C11 (GPa): stiffness
        C33 (GPa): stiffness
        C13 (GPa): stiffness
        C44 (GPa): stiffness
        C65 (GPa): stiffness

    Returns:
        C: 6x6 stiffness matrix
    """   
    C12= C11-2*C66
    C=np.array([[C11,C12,C13,0,0,0],
            [C12,C11,C13,0,0,0],
            [C13,C13,C33,0,0,0],
            [0,0,0,C44,0,0],
            [0,0,0,0,C44,0],
            [0,0,0,0,0,C66]])
    return C

In [ ]:
def zoeppritz(vp1, vs1, rho1, vp2, vs2, rho2, theta):

    '''
    Reflection & Transmission coefficients calculated using full Zoeppritz
    equations.
    
    Usage:
    ------
    R = rc_zoep(vp1, vs1, rho1, vp2, vs2, rho2, theta1)
    
    Reference:
    ----------
    The Rock Physics Handbook, Dvorkin et al.
    '''
    
    # Calculate reflection & transmission angles
    theta1 = np.deg2rad(theta)
   
    
    p = np.sin(theta)/vp1 # Ray parameter
    # Transmission angle of P-wave
    theta2  =np.arcsin(p*vp2)
    phi1   = np.arcsin(p*vs1);      # Reflection angle of converted S-wave
    phi2   = np.arcsin(p*vs2);      # Transmission angle of converted S-wave
    
    # Matrix form of Zoeppritz Equations... M & N are two of the matricies
    M = np.array([ 
        [-np.sin(theta1), -np.cos(phi1), np.sin(theta2), np.cos(phi2)],
        [np.cos(theta1), -np.sin(phi1), np.cos(theta2), -np.sin(phi2)],
        [2*rho1*vs1*np.sin(phi1)*np.cos(theta1), rho1*vs1*(1-2*np.sin(phi1)**2),
            2*rho2*vs2*np.sin(phi2)*np.cos(theta2), rho2*vs2*(1-2*np.sin(phi2)**2)],
        [-rho1*vp1*(1-2*np.sin(phi1)**2), rho1*vs1*np.sin(2*phi1), 
            rho2*vp2*(1-2*np.sin(phi2)**2), -rho2*vs2*np.sin(2*phi2)]
        ])
    
    N = np.array([ 
        [np.sin(theta1), np.cos(phi1), -np.sin(theta2), -np.cos(phi2)],
        [np.cos(theta1), -np.sin(phi1), np.cos(theta2), -np.sin(phi2)],
        [2*rho1*vs1*np.sin(phi1)*np.cos(theta1), rho1*vs1*(1-2*np.sin(phi1)**2),
            2*rho2*vs2*np.sin(phi2)*np.cos(theta2), rho2*vs2*(1-2*np.sin(phi2)**2)],
        [rho1*vp1*(1-2*np.sin(phi1)**2), -rho1*vs1*np.sin(2*phi1),
            -rho2*vp2*(1-2*np.sin(phi2)**2), rho2*vs2*np.sin(2*phi2)]
        ])
    a=rho2*(1-2*np.sin(phi2)**2)-rho1*(1-2*np.sin(phi1)**2)
    b=rho2*(1-2*np.sin(phi2)**2)+2*rho1*np.sin(phi1)**2
    c=rho1*(1-2*np.sin(phi1)**2)+2*rho2*np.sin(phi2)**2
    d=2*(rho2*vs2**2-rho1*vs1**2)
    H=a-d*np.cos(theta2)/vp2*np.cos(phi2)/vs2
    E=b*np.cos(theta1)/vp1+c*np.cos(theta2)/vp2
    F=b*np.cos(phi1)/vs1+c*np.cos(phi2)/vs2
    G=a-d*np.cos(theta1)/vp1*np.cos(phi2)/vs2
    D=E*F+G*H*p**2
    Rpp=( (  b*np.cos(theta1)/vp1-c*np.cos(theta2)/vp2)*F-( a+d*np.cos(theta1)/vp1*np.cos(phi2)/vs2)*H*p**2)/D

    # This is the important step, calculating coefficients for all modes and rays

    
    
    #R = np.dot(np.linalg.inv(M), N)
    
    return Rpp

In [11]:
theta= np.radians(30)
vsvp=0.5
x1=2*np.sin(theta)**2-1-2*np.cos(theta)*np.sqrt(vsvp**2-np.sin(theta)**2)
a=np.tan(theta)*x1/vsvp
a

-0.577350284090787

In [8]:
np.sin(np.radians(30))

0.49999999999999994

In [38]:
theta= np.radians(30)
SP=0.5
b=np.sin(theta)/np.sqrt(1-SP**2*np.sin(theta)**2)*(4*np.sin(theta)**2*SP**2-4*SP*np.cos(theta)*np.sqrt(1-SP**2*np.sin(theta)**2))
b

-0.736925958910858

In [39]:
theta= np.radians(30)
vsvp=0.5
x2=np.sin(theta)**2-np.cos(theta)*np.sqrt(vsvp**2-np.sin(theta)**2)
b=4*np.tan(theta)*x2/vsvp
b

1.1547005085769289

In [40]:
vpvs=1/vsvp
facroo=np.sqrt(vpvs**2-np.sin(theta)**2)
facall=np.sin(theta)/(vpvs*facroo)
b=4.*facall*(np.sin(theta)**2 - np.cos(theta)*facroo)
b

-0.736925958910858

In [35]:
theta= np.radians(50)
vsvp=1.5
print(np.tan(theta)/vsvp)
print(np.sin(theta)/np.sqrt(1-SP**2*np.sin(theta)**2))

0.7945023950628066
1.1917535925942098


0.5773502691896257